In [26]:
#Some preprocessing done in my Analysis file

import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
weapons = pd.read_csv("C:/Users/jaram/DS/Capstone/weapons.csv")
weapons = weapons.drop(columns = ['Unnamed: 0', 'Trend'])
weapons = weapons[weapons.Score != 0.0]
import re 
def fix_weapons(a):
    return a[len(a)//2:]


weapons.Weapon = weapons.Weapon.apply(fix_weapons)
weapons.Weapon.unique()
weapons = weapons[weapons.Rank != 'iron'] #No iron data 

#More type cleaning 
def p2f(x):
    return float(x.strip('%'))

weapons['Win %'] =weapons['Win %'].apply(p2f)
weapons['Pick %'] =weapons['Pick %'].apply(p2f)

#Make tiering binary for modeling purposes 
def set_tiers(tier):
    if tier in ['Fair / B', 'Weak / C', 'Good / A','Bad / D']:
        return(0)

    else:
        return(1)
weapons['Tier'] = weapons['Tier'].apply(set_tiers)
weapons.head()


,Weapon,Category,Cost,Tier,Score,Win %,Pick %,Dmg/Round,KDA,Patch,Rank
18,Ares,HEAVY,1600,0,45.90,47.22,2.33,79.60,1.17,1.14,silver
19,Bucky,SHOTGUN,900,0,32.45,45.25,1.84,58.11,0.87,1.14,silver
20,Bulldog,RIFLE,2100,0,44.79,47.46,1.21,78.48,1.09,1.14,silver
21,Classic,SIDEARM,0,0,21.39,41.57,9.12,42.72,0.80,1.14,silver
22,Frenzy,SIDEARM,400,0,44.91,47.94,1.55,66.80,1.15,1.14,silver


In [27]:
weapons.groupby('Patch').count()

,Weapon,Category,Cost,Tier,Score,Win %,Pick %,Dmg/Round,KDA,Rank
Patch,,,,,,,,,,
1.14,108,108,108,108,108,108,108,108,108,108
2.11,108,108,108,108,108,108,108,108,108,108
3.12,108,108,108,108,108,108,108,108,108,108
4.00,107,107,107,107,107,107,107,107,107,107
4.01,108,108,108,108,108,108,108,108,108,108
4.02,108,108,108,108,108,108,108,108,108,108
4.03,108,108,108,108,108,108,108,108,108,108
4.04,108,108,108,108,108,108,108,108,108,108
4.05,108,108,108,108,108,108,108,108,108,108


In [28]:
#Modeling, choosing three models
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
from sklearn.ensemble import GradientBoostingClassifier 
gbc  = GradientBoostingClassifier()

In [29]:
weapons.dtypes

Weapon        object
Category      object
Cost           int64
Tier           int64
Score        float64
Win %        float64
Pick %       float64
Dmg/Round    float64
KDA          float64
Patch        float64
Rank          object
dtype: object

In [30]:
weapons = pd.get_dummies(weapons, columns=["Weapon", "Category", "Rank"]) #One hot encoding for categorical variables 


In [31]:
weapons.Tier.value_counts()

0    2160
1     323
Name: Tier, dtype: int64

In [32]:
from sklearn.utils import resample
df_majority = weapons[weapons.Tier == 0]
df_minority = weapons[weapons.Tier == 1]

df_majority_downsample = resample(df_majority, 
                                 replace=True,     
                                 n_samples=323,    
                                 random_state=42) 
df_train = pd.concat([df_majority_downsample, df_minority])

 
# Display new class counts
print (df_train.Tier.value_counts())

0    323
1    323
Name: Tier, dtype: int64


In [33]:
#Split Data
from sklearn.model_selection import train_test_split
X = df_train.drop(columns = ['Tier'])
y = df_train.Tier 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3) #split
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(452, 38)
(194, 38)
(452,)
(194,)


# Train Models, Print Results

### Under Sampling Method

In [34]:
#Decision Tree Classifier 
from sklearn.metrics import classification_report
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
target_names = ['0', '1']
print(classification_report(y_test, y_pred,target_names=target_names))


              precision    recall  f1-score   support

           0       0.94      1.00      0.97       106
           1       1.00      0.92      0.96        88

    accuracy                           0.96       194
   macro avg       0.97      0.96      0.96       194
weighted avg       0.97      0.96      0.96       194



In [35]:
#Random Forest Classifier
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
target_names = ['0', '1']
print(classification_report(y_test, y_pred,target_names=target_names))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       106
           1       1.00      0.97      0.98        88

    accuracy                           0.98       194
   macro avg       0.99      0.98      0.98       194
weighted avg       0.98      0.98      0.98       194



In [36]:
#Gradient Boosted Classifier 
gbc.fit(X_train, y_train)
y_pred = gbc.predict(X_test)
target_names = ['0', '1']
print(classification_report(y_test, y_pred,target_names=target_names))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98       106
           1       1.00      0.94      0.97        88

    accuracy                           0.97       194
   macro avg       0.98      0.97      0.97       194
weighted avg       0.98      0.97      0.97       194



### Over Sampling Method

In [37]:
df_majority = weapons[weapons.Tier == 0]
df_minority = weapons[weapons.Tier == 1]

df_minority_downsample = resample(df_minority, 
                                 replace=True,     
                                 n_samples=2160,    
                                 random_state=42) 
df_train = pd.concat([df_minority_downsample, df_majority])

 
# Display new class counts
print (df_train.Tier.value_counts())

1    2160
0    2160
Name: Tier, dtype: int64


In [38]:
#Decision Tree Classifier 
from sklearn.metrics import classification_report
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
target_names = ['0', '1']
print(classification_report(y_test, y_pred,target_names=target_names))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       106
           1       1.00      0.92      0.96        88

    accuracy                           0.96       194
   macro avg       0.97      0.96      0.96       194
weighted avg       0.97      0.96      0.96       194



In [39]:
#Random Forest Classifier
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
target_names = ['0', '1']
print(classification_report(y_test, y_pred,target_names=target_names))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       106
           1       1.00      0.97      0.98        88

    accuracy                           0.98       194
   macro avg       0.99      0.98      0.98       194
weighted avg       0.98      0.98      0.98       194



In [40]:
#Gradient Boosted Classifier 
gbc.fit(X_train, y_train)
y_pred = gbc.predict(X_test)
target_names = ['0', '1']
print(classification_report(y_test, y_pred,target_names=target_names))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98       106
           1       1.00      0.94      0.97        88

    accuracy                           0.97       194
   macro avg       0.98      0.97      0.97       194
weighted avg       0.98      0.97      0.97       194



## Best Model : Over Sampled Random forest classifier 

In [50]:
for i in range(1, 19):
    arr = np.array(X_test.iloc[i])
    arr = arr.reshape(-1, 38)
    print(rfc.predict(arr))


#Model predicts that the Vandal, and the Marshal need tuning   

[1]
[1]
[0]
[0]
[1]
[1]
[1]
[1]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]


C:\Users\jaram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\jaram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\jaram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\jaram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:4

In [57]:
X_test.iloc[17]

Cost                     950.00
Score                     40.43
Win %                     45.13
Pick %                     2.68
Dmg/Round                 71.22
KDA                        0.89
Patch                      5.05
Weapon_Ares                0.00
Weapon_Bucky               0.00
Weapon_Bulldog             0.00
Weapon_Classic             0.00
Weapon_Frenzy              0.00
Weapon_Ghost               0.00
Weapon_Guardian            0.00
Weapon_Judge               0.00
Weapon_Marshal             1.00
Weapon_Odin                0.00
Weapon_Operator            0.00
Weapon_Phantom             0.00
Weapon_Sheriff             0.00
Weapon_Shorty              0.00
Weapon_Spectre             0.00
Weapon_Stinger             0.00
Weapon_Tactical Knife      0.00
Weapon_Vandal              0.00
Category_HEAVY             0.00
Category_MELEE             0.00
Category_RIFLE             0.00
Category_SHOTGUN           0.00
Category_SIDEARM           0.00
Category_SMG               0.00
Category